## Procesamiento de Datos

![logo](img/logo.jpeg)


# Sumario
- Trabajo con strings
- Combinando datasets
- Limpieza de datos
 - map, filter, reduce
 - filling missing values
 - valores duplicados
 - categorizacion de datos

In [1]:
import numpy as np
import pandas as pd

### Combinar varios datasets 
- En base a un elemento en común (índice)
- MovieLens 'UserId'

In [2]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os

# descargar MovieLens dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'  
local_zip = os.path.join("res", "ml-1m.zip")
urllib.request.urlretrieve(url, local_zip)
# descomprimiendo archivo zip
with zp.ZipFile(local_zip, 'r') as zipp: 
    print('Extracting all files...') 
    zipp.extractall(os.path.join("res")) # destino
    print('Done!') 

Extracting all files...
Done!


In [3]:
root_path = os.path.join("res", "ml-1m" )

ratings_dataset = pd.read_csv(os.path.join(root_path, "ratings.dat"), sep='::',
                                index_col=0, engine='python',
                                names=['UserID','MovieID','Rating','Timestamp'])

users_dataset = pd.read_csv(os.path.join(root_path, "users.dat"),sep='::',
                              index_col=0, engine='python',
                              names=['UserID','Gender','Age','Occupation','Zip-code'])

In [4]:
users_dataset.sample(5)

,Gender,Age,Occupation,Zip-code
UserID,,,,
3691,M,25,7,21030
2904,M,25,4,16802
5628,M,18,4,90024
42,M,25,8,24502
4096,M,25,15,60004


In [5]:
ratings_dataset.sample(5)

,MovieID,Rating,Timestamp
UserID,,,
5848,2004,1,957784984
5493,1888,2,959813722
520,339,4,976226968
5763,3423,1,959537670
1135,3896,3,974879914


### Uniendo datasets con 'join' y 'merge'
- merge() == join()
 - 'join' utiliza por defecto los índices para unir
- Utilizando el parámetro 'on'
 - Si las columnas difieren, 'left_on' y 'right_on'
 
 https://i.stack.imgur.com/hMKKt.jpg

In [6]:
# combinando users y ratings, ¿Cómo?
combined_dataset = users_dataset.merge(ratings_dataset, on='UserID', how='inner') # parametro 'on' define la columna pivote
display(combined_dataset.sample(5))
len(combined_dataset)

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp
UserID,,,,,,,
3681,M,25,7,29662,2997,4,966534293
5561,F,25,3,74075,293,4,959374671
4802,M,56,1,40601,3654,5,962977879
3372,M,56,13,55118,1645,4,967604716
1242,F,35,0,10014,1477,3,974908596


1000209

In [7]:
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::',encoding='latin-1', engine='python',names=['MovieID','Title','Genre'])
movies_dataset.sample(5)

,MovieID,Title,Genre
2904,2973,Crimes and Misdemeanors (1989),Comedy
1644,1690,Alien: Resurrection (1997),Action|Horror|Sci-Fi
3379,3448,"Good Morning, Vietnam (1987)",Comedy|Drama|War
1678,1727,"Horse Whisperer, The (1998)",Drama
3111,3180,Play it to the Bone (1999),Comedy|Drama


In [8]:
# combinando movies y el resto
all_dataset = combined_dataset.merge(movies_dataset,on='MovieID', how='inner')
all_dataset.sample(5)

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
723218,M,50,2,33134,908,4,965238479,North by Northwest (1959),Drama|Thriller
905796,F,25,14,07030,587,3,959866960,Ghost (1990),Comedy|Romance|Thriller
928607,M,25,0,12833,971,5,959188568,Cat on a Hot Tin Roof (1958),Drama
194454,M,50,13,77459,1957,5,974920535,Chariots of Fire (1981),Drama
315732,M,35,0,91006,2971,4,975382824,All That Jazz (1979),Musical


### Concatenate
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

## Pivot
- Representar los datos en función a varios parámetros, agregando
```python
pivot_table(<lista de valores>, index=<agregador primario>, columns=<agregador secundario>)
```
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

In [9]:
# all_dataset.pivot_table('Rating', index='Gender', columns='Age')
# all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc='count')
all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc=['count', 'mean']) # cuenta por sexo y edad

count                                                   mean  \
Age        1       18      25      35     45     50     56        1    
Gender                                                                 
F        8827   45427   91340   49473  24110  18064   9199  3.616291   
M       18384  138109  304216  149530  59523  54426  29581  3.517461   

                                                                   
Age           18       25        35        45        50        56  
Gender                                                             
F       3.453145  3.60670  3.659653  3.663044  3.797110  3.915534  
M       3.525476  3.52678  3.604434  3.627942  3.687098  3.720327

## Agrupaciones
- agg -> funciones estadísticas de agregación
- Series.unique() -> valores únicos
- pd.value_counts -> ocurrencias

## Manipulación de strings
```python
split(): separar en bloques en función de un carácter
replace(): reemplazar un carácter por otro
index(): encontrar la posición de un carácter
```

In [10]:
# Ejemplo con MovieLens: Genre
## 1: obtener todos los géneros por separado
## 2: crear un dataset de géneros
## 3: por película, marcar género por separado
## 4: unir con dataset original
movies_dataset.head(3)

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [11]:
all_genres = movies_dataset['Genre'].apply(lambda x : x.split('|'))
print(all_genres)

# print([genre for x in all_genres for genre in x])

genres = pd.unique([genre for x in all_genres for genre in x])
display(genres)

0        [Animation, Children's, Comedy]
1       [Adventure, Children's, Fantasy]
2                      [Comedy, Romance]
3                        [Comedy, Drama]
4                               [Comedy]
                      ...               
3878                            [Comedy]
3879                             [Drama]
3880                             [Drama]
3881                             [Drama]
3882                   [Drama, Thriller]
Name: Genre, Length: 3883, dtype: object


/tmp/ipykernel_7840/456606724.py:6: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  genres = pd.unique([genre for x in all_genres for genre in x])


array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [12]:
# crear tabla con columnas por género
zeros = np.zeros( (len(movies_dataset), len(genres)) )
genres_frame = pd.DataFrame(zeros, columns=genres)
genres_frame.head(3)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
columns_genres = genres_frame.columns # lista de generos (columnas)
# para cada película, marcar género con 1
for i, genre in enumerate(movies_dataset['Genre']):
    inds = columns_genres.get_indexer(genre.split('|')) # retorna los indices correspondientes a los generos de cada pelicula
    genres_frame.iloc[i,inds] = 1 # localiza las columnas del genero correspondiente, marca con 1

In [14]:
genres_frame.head(5)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# unir con dataset original
movies_split_genre = movies_dataset.join(genres_frame)

In [16]:
display(movies_split_genre.head(5))

,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Replace e index para extraer el año de la película

In [17]:
movies_dataset.head(2)

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


In [18]:
# extraer el año de la columna Title
def split_year(title):
    index = title.index('(')  # establecemos el 'punto de partida' para sacar el año
    return title[index:].replace('(','').replace(')','') #sustituimos los paréntesis por 'nada'
    
# crear nueva columna Year
movies_dataset['Year'] = movies_dataset['Title'].apply(split_year)
display(movies_dataset.sample(2))

,MovieID,Title,Genre,Year
381,385,"Man of No Importance, A (1994)",Drama,1994
3166,3235,Where the Buffalo Roam (1980),Comedy,1980


In [19]:
# eliminar el año de la columna Title
def remove_year(title):
    index = title.index('(')
    return title[:index-1].strip()

movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year)
movies_dataset.head(2)

,MovieID,Title,Genre,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995


#### Expresiones regulares
https://docs.python.org/3/library/re.html

- import re

In [20]:
# ¿Cómo localizar que 'Zip-code' tiene un formato erróneo?
users_dataset.sample(5)

# users_dataset['Zip-code'].str.match('^\d{5}$')

users_dataset[users_dataset['Zip-code'].str.match('^\d{5}$') == False]

# ^\d{5}$
# ^ = start of the string
# \d = decimal string
# {5} = 5 repeticiones de decimales
# $ = end of string

,Gender,Age,Occupation,Zip-code
UserID,,,,
161,M,45,16,98107-2117
233,F,45,20,37919-4204
293,M,56,1,55337-4056
458,M,50,16,55405-2546
506,M,25,16,55103-1006
...,...,...,...,...
5682,M,18,0,23455-4959
5904,F,45,12,954025
5925,F,25,0,90035-4444


In [21]:
# ¿Cómo extraer el año con regex en el formato adecuado?
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python',encoding='latin-1',names=['MovieID','Title','Genre'])
display(movies_dataset.head(2))
movies_dataset['Title'].str.extract('(\d{4})')

# (\d{4})
# (= busca apertura parentesis
# \d = decimal string
# {4} = 4 repeticiones de decimales
# ) = cierre de parentesis

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


,0
0,1995
1,1995
2,1995
3,1995
4,1995
...,...
3878,2000
3879,2000
3880,2000
3881,2000


## Operaciones con colecciones
```python
reduce: aplicar una operación y retornar un valor
map: aplicar  una operación y retornar una secuencia
filter: retorna una secuencia con elementos que cumplen una condición
```


## Reduce
- Aplicar una operación matemática a cada uno de los elementos de una colección
- Diferente de 'apply()' porque retorna un valor numérico
- Ejemplo: Detección de géneros en años específicos

https://docs.python.org/3/library/functools.html

In [22]:
from functools import reduce # necesario para reduce

lista = [1, 3, 5, 7, 9]
print(reduce(lambda x,y: x + y, lista))

25


In [23]:
movies_1975 = movies_split_genre[ movies_split_genre['Title'].str.contains('1975') ]
movies_1975.head(3)

,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
707,716,Switchblade Sisters (1975),Crime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
994,1007,"Apple Dumpling Gang, The (1975)",Children's|Comedy|Western,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
996,1009,Escape to Witch Mountain (1975),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
any_drama = reduce(lambda x,y : bool(x) | bool(y),movies_1975['Drama']) # hay algún drama en 1975
print(any_drama)

all_comedy = reduce(lambda x,y : bool(x) & bool(y),movies_1975['Comedy']) # son todas las películas de 1975 comedias?
print(all_comedy)

True
False


In [25]:
print(movies_1975['Drama'].any()) # Comprueba si hay algún valor que puede cumplir  
print(movies_1975['Comedy'].all()) # Comprueba si todos los valores son True

True
False


In [26]:
# Observar el tipo de dato antes para ver si es posible aplicar las funciones
print(movies_1975.dtypes)
print(movies_1975['Comedy'].unique())

MovieID          int64
Title           object
Genre           object
Animation      float64
Children's     float64
Comedy         float64
Adventure      float64
Fantasy        float64
Romance        float64
Drama          float64
Action         float64
Crime          float64
Thriller       float64
Horror         float64
Sci-Fi         float64
Documentary    float64
War            float64
Musical        float64
Mystery        float64
Film-Noir      float64
Western        float64
dtype: object
[0. 1.]


## Filter
- retorna una secuencia con elementos que cumplen una condición
- Ejemplo: obtener las películas de 1975 que contienen 'The' en el título

In [27]:
filtro = filter(lambda x : 'The' in x, movies_1975['Title'])
list(filtro)
# ¿Están todos los títulos con "The"? si tiene mayúsculas o no...

['Apple Dumpling Gang, The (1975)',
 'Man Who Would Be King, The (1975)',
 'Stepford Wives, The (1975)',
 'Rocky Horror Picture Show, The (1975)',
 'McCullochs, The (1975)',
 'Mirror, The (Zerkalo) (1975)']

## Map
- aplicar  una operación y retornar una secuencia
- Cambiar el valor integral de la columna 'Comedy' por bool

In [28]:
mapa = map(lambda x : bool(x), movies_1975['Comedy'])
movies_1975.loc[:,'Comedy'] = list(mapa)
movies_1975.head(4)

/tmp/ipykernel_7840/4198280980.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False, True, False, False, False, False, False, True, False, False, False, True, False, False, False, False, True, False, True, False, False, True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  movies_1975.loc[:,'Comedy'] = list(mapa)


,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
707,716,Switchblade Sisters (1975),Crime,0.0,0.0,False,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
994,1007,"Apple Dumpling Gang, The (1975)",Children's|Comedy|Western,0.0,1.0,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
996,1009,Escape to Witch Mountain (1975),Adventure|Children's|Fantasy,0.0,1.0,False,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1176,1193,One Flew Over the Cuckoo's Nest (1975),Drama,0.0,0.0,False,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transformación de variables (calidad de datos)
- Valores no definidos
- Valores duplicados
- Discretización (valores categóricos)

In [29]:
matrix = pd.DataFrame(np.random.randint(10,size=(5,10)))
matrix[matrix < 2] = np.nan
matrix

,0,1,2,3,4,5,6,7,8,9
0,NaN,3.0,NaN,NaN,5.0,9.0,9.0,2.0,3.0,5.0
1,7.0,2.0,NaN,2.0,3.0,8.0,4.0,8.0,5.0,NaN
2,9.0,NaN,5.0,NaN,9.0,NaN,9.0,NaN,8.0,NaN
3,NaN,9.0,6.0,5.0,8.0,7.0,2.0,2.0,NaN,6.0
4,2.0,NaN,NaN,6.0,NaN,NaN,NaN,4.0,NaN,8.0


In [30]:
# nulos por columna
matrix.isnull().sum()
# matrix.isna().sum()

0    2
1    2
2    3
3    2
4    1
5    2
6    1
7    1
8    2
9    2
dtype: int64

In [31]:
# Cantidad valores nulos
matrix.isnull().sum().sum()

np.int64(18)

In [32]:
# numero de no nulos por fila
matrix.count(axis=1)

0    7
1    8
2    5
3    8
4    4
dtype: int64

In [33]:
# Número de nulos por fila
matrix.shape[1] - matrix.count(axis=1)

0    3
1    2
2    5
3    2
4    6
dtype: int64

In [34]:
# Representación de las filas en las que una determinada columna tiene nulos
matrix[matrix[1].isnull()]

,0,1,2,3,4,5,6,7,8,9
2,9.0,NaN,5.0,NaN,9.0,NaN,9.0,NaN,8.0,NaN
4,2.0,NaN,NaN,6.0,NaN,NaN,NaN,4.0,NaN,8.0


In [35]:
# Conteo de valores que aparecen en el dataset
valores = [8, 4]
# Identificación de valores de dominio que se encuentran en un listado
matrix[matrix[6].isin(valores)]

,0,1,2,3,4,5,6,7,8,9
1,7.0,2.0,NaN,2.0,3.0,8.0,4.0,8.0,5.0,NaN


In [36]:
## Tratamiento de valores nulos
# eliminar
matrix.dropna()

,0,1,2,3,4,5,6,7,8,9


In [37]:
# eliminar si no hay un número de valores no NaN
matrix.dropna(thresh=7)

,0,1,2,3,4,5,6,7,8,9
0,NaN,3.0,NaN,NaN,5.0,9.0,9.0,2.0,3.0,5.0
1,7.0,2.0,NaN,2.0,3.0,8.0,4.0,8.0,5.0,NaN
3,NaN,9.0,6.0,5.0,8.0,7.0,2.0,2.0,NaN,6.0


In [38]:
# sustituir por un valor fijo
matrix.fillna(-1)

,0,1,2,3,4,5,6,7,8,9
0,-1.0,3.0,-1.0,-1.0,5.0,9.0,9.0,2.0,3.0,5.0
1,7.0,2.0,-1.0,2.0,3.0,8.0,4.0,8.0,5.0,-1.0
2,9.0,-1.0,5.0,-1.0,9.0,-1.0,9.0,-1.0,8.0,-1.0
3,-1.0,9.0,6.0,5.0,8.0,7.0,2.0,2.0,-1.0,6.0
4,2.0,-1.0,-1.0,6.0,-1.0,-1.0,-1.0,4.0,-1.0,8.0


In [39]:
# sustituir por valor dinámico (copia)
print(matrix)
matrix.fillna(method='bfill') # bfill y ffill

     0    1    2    3    4    5    6    7    8    9
0  NaN  3.0  NaN  NaN  5.0  9.0  9.0  2.0  3.0  5.0
1  7.0  2.0  NaN  2.0  3.0  8.0  4.0  8.0  5.0  NaN
2  9.0  NaN  5.0  NaN  9.0  NaN  9.0  NaN  8.0  NaN
3  NaN  9.0  6.0  5.0  8.0  7.0  2.0  2.0  NaN  6.0
4  2.0  NaN  NaN  6.0  NaN  NaN  NaN  4.0  NaN  8.0


/tmp/ipykernel_7840/1491580683.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  matrix.fillna(method='bfill') # bfill y ffill


,0,1,2,3,4,5,6,7,8,9
0,7.0,3.0,5.0,2.0,5.0,9.0,9.0,2.0,3.0,5.0
1,7.0,2.0,5.0,2.0,3.0,8.0,4.0,8.0,5.0,6.0
2,9.0,9.0,5.0,5.0,9.0,7.0,9.0,2.0,8.0,6.0
3,2.0,9.0,6.0,5.0,8.0,7.0,2.0,2.0,NaN,6.0
4,2.0,NaN,NaN,6.0,NaN,NaN,NaN,4.0,NaN,8.0


In [40]:
# sustituir por valor dinámico (interpolación)
print(matrix)
matrix.interpolate()

     0    1    2    3    4    5    6    7    8    9
0  NaN  3.0  NaN  NaN  5.0  9.0  9.0  2.0  3.0  5.0
1  7.0  2.0  NaN  2.0  3.0  8.0  4.0  8.0  5.0  NaN
2  9.0  NaN  5.0  NaN  9.0  NaN  9.0  NaN  8.0  NaN
3  NaN  9.0  6.0  5.0  8.0  7.0  2.0  2.0  NaN  6.0
4  2.0  NaN  NaN  6.0  NaN  NaN  NaN  4.0  NaN  8.0


,0,1,2,3,4,5,6,7,8,9
0,NaN,3.0,NaN,NaN,5.0,9.0,9.0,2.0,3.0,5.000000
1,7.0,2.0,NaN,2.0,3.0,8.0,4.0,8.0,5.0,5.333333
2,9.0,5.5,5.0,3.5,9.0,7.5,9.0,5.0,8.0,5.666667
3,5.5,9.0,6.0,5.0,8.0,7.0,2.0,2.0,8.0,6.000000
4,2.0,9.0,6.0,6.0,8.0,7.0,2.0,4.0,8.0,8.000000


#### Tratar valores duplicados

In [41]:
serie = pd.Series(['a','b','c','a','c','a','g'])
serie.duplicated()

0    False
1    False
2    False
3     True
4     True
5     True
6    False
dtype: bool

In [42]:
df = all_dataset
df
# eliminar
# Eliminación de los duplicados en una columna definida
df2 = df.drop_duplicates(subset="Gender", keep='last', inplace=False)
display(df2)

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
999867,F,45,0,01060,1097,4,956705811,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1000208,M,25,6,11106,1097,4,956715569,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


#### Discretización (valores categóricos)
- Tras Series y DataFrame, objeto para categorías: Categorical
```python
categorias = pd.cut(<valores>, <bins>) 
```

In [43]:
# especificar los bloques
bins = [0,18,35,65,99]
edades = [16,25,18,71,44,100,12]
categorias = pd.cut(edades,bins)
print(categorias)

[(0.0, 18.0], (18.0, 35.0], (0.0, 18.0], (65.0, 99.0], (35.0, 65.0], NaN, (0.0, 18.0]]
Categories (4, interval[int64, right]): [(0, 18] < (18, 35] < (35, 65] < (65, 99]]


In [44]:
categorias.value_counts()

(0, 18]     3
(18, 35]    1
(35, 65]    1
(65, 99]    1
Name: count, dtype: int64

In [45]:
# especificar el número de bloques
bins = 5
edades = [0,6,8,16,25,18,71,44,100]
categorias = pd.cut(edades,bins) # rangos idénticos (similar distancia de rangos)
print(categorias)
print(categorias.value_counts())

[(-0.1, 20.0], (-0.1, 20.0], (-0.1, 20.0], (-0.1, 20.0], (20.0, 40.0], (-0.1, 20.0], (60.0, 80.0], (40.0, 60.0], (80.0, 100.0]]
Categories (5, interval[float64, right]): [(-0.1, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]
(-0.1, 20.0]     5
(20.0, 40.0]     1
(40.0, 60.0]     1
(60.0, 80.0]     1
(80.0, 100.0]    1
Name: count, dtype: int64


In [46]:
bins = 5
edades = [1,6,8,16,25,18,71,44,100]
categorias = pd.qcut(edades,bins) # rangos homogéneos (similar número de valores)
print(categorias)
print(categorias.value_counts())

[(0.999, 7.2], (0.999, 7.2], (7.2, 16.4], (7.2, 16.4], (23.6, 54.8], (16.4, 23.6], (54.8, 100.0], (23.6, 54.8], (54.8, 100.0]]
Categories (5, interval[float64, right]): [(0.999, 7.2] < (7.2, 16.4] < (16.4, 23.6] < (23.6, 54.8] < (54.8, 100.0]]
(0.999, 7.2]     2
(7.2, 16.4]      2
(16.4, 23.6]     1
(23.6, 54.8]     2
(54.8, 100.0]    2
Name: count, dtype: int64


## <img src="img/by-nc.png" width="200">